In [1]:
import numpy as np

# Time Wrapper

In [2]:
import time
from functools import wraps


def timing_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        total_execution_time = 0
        iterations = 100000

        for _ in range(iterations):
            start_time = time.perf_counter()
            result = func(*args, **kwargs)
            end_time = time.perf_counter()
            total_execution_time += (end_time - start_time)

        average_execution_time = total_execution_time / iterations
        print(
            f'Average execution time over {iterations} iterations: {average_execution_time:.8f} seconds')

        return result

    return wrapper

## Mean|

In [3]:
@timing_decorator
def custom_mean(x):
    return np.sum(x)/x.size


@timing_decorator
def mean(x):
    return np.mean(x)

In [4]:
arr = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [ ]:
custom_mean(arr)

In [ ]:
mean(arr)

# Var

In [ ]:
def custom_mean(x):
    return np.sum(x)/x.size

In [ ]:
@timing_decorator
def custom_var(x):
    mean_x = custom_mean(x)
    return custom_mean(x**2) - mean_x**2


@timing_decorator
def custom_var_2(x):
    return custom_mean(x**2) - custom_mean(x)**2


@timing_decorator
def custom_var_3(x):
    return custom_mean(np.square(x-custom_mean(x)))


@timing_decorator
def var(x):
    return np.var(x)

In [ ]:
@timing_decorator
def custom_var_optimized(x):
    x_squared_sum = np.sum(x**2)
    x_sum = np.sum(x)
    x_size = x.size
    return (x_squared_sum / x_size) - (x_sum / x_size)**2

In [ ]:
@timing_decorator
def custom_unbias_var_optimized(x):
    x_squared_sum = np.sum(x**2)
    x_sum = np.sum(x)
    x_size = x.size-1
    return (x_squared_sum / x_size) - (x_sum / x_size)**2

In [ ]:
custom_unbias_var_optimized(arr)

In [ ]:
custom_var(arr)

In [ ]:
custom_var_optimized(arr)

In [ ]:
custom_var_2(arr)

In [ ]:
custom_var_3(arr)

In [ ]:
var(arr)

# Std

In [ ]:
def custom_var_optimized(x):
    x_squared_sum = np.sum(x**2)
    x_sum = np.sum(x)
    x_size = x.size-1
    return (x_squared_sum / x_size) - (x_sum / x_size)**2

In [ ]:
@timing_decorator
def custom_std(x):
    return np.sqrt(custom_var_optimized(x))


@timing_decorator
def std(x):
    return np.std(x)

In [ ]:
std(arr)

In [ ]:
custom_std(arr)

# Median

In [ ]:
@timing_decorator
def custom_median(x):
    x_sort = np.sort(x)
    n = x.size
    if n%2 == 1:
        return x_sort[n//2]
    else:
        m1 = x_sort[n//2]
        m2 = x_sort[n//2-1]
        return (m1+m2)/2

In [ ]:
@timing_decorator
def median(x):
    return np.median(x)

In [ ]:
median(arr)

In [ ]:
custom_median(arr)

# Mode

In [ ]:
@timing_decorator
def custom_mode(x):

    assert len(x.shape) == 1

    counts = np.bincount(x)

    mode_value = np.argmax(counts)

    return mode_value

In [ ]:
from scipy import stats

@timing_decorator
def mode(x):
    return stats.mode(x)

In [ ]:
# mode(arr)

In [ ]:
custom_mode(arr)

# Percentile

In [ ]:
# @timing_decorator
def custom_percentile(x, percentile):
    # Step 1: Sorting the data
    x_sort = np.sort(x)  
    
    # Step 2: Finding the index of the percentile value
    index = (percentile / 100) * (x_sort.size - 1)  
    
    # Step 3: Interpolating to get the percentile value
    lower = np.floor(index).astype(int)  # Finding the lower index
    upper = np.ceil(index).astype(int)   # Finding the upper index
    if lower == upper:  # If index is an integer
        return x_sort[lower]  # Return the value at index
    else:
        # Interpolate between the values at the lower and upper indices
        return x_sort[lower] + (x_sort[upper] - x_sort[lower]) * (index - lower)


In [ ]:
@timing_decorator
def percentile(x, percentile):
    return np.percentile(x, percentile)

In [ ]:
custom_percentile(arr, percentile=20)

In [ ]:
percentile(arr, percentile=20)

# Covariance 

In [ ]:
arr1 = np.array([1, 2, 3, 4, 5])
arr2 = np.array([6, 7, 8, 9, 10])

In [ ]:
def custom_mean(x):
    return np.sum(x) / x.size

def custom_var(x):
    mean_x = custom_mean(x)
    return np.sum((x - mean_x)**2) / (x.size - 1)
@timing_decorator
def custom_cov(x, y):
    mean_x = custom_mean(x)
    mean_y = custom_mean(y)
    cov_xy = np.sum((x - mean_x) * (y - mean_y)) / (x.size - 1)
    var_x = custom_var(x)
    var_y = custom_var(y)
    cov_matrix = np.array([[var_x, cov_xy], [cov_xy, var_y]])
    return cov_matrix

In [ ]:
@timing_decorator
def cov(x,y):
    return np.cov(x ,y)

In [ ]:
custom_cov_matrix(arr1, arr2)

In [ ]:
custom_cov(arr1, arr2)

In [ ]:
cov(arr1, arr2)